In [9]:
import numpy as np 
import pandas as pd 
import math
import os
import tensorflow as tf
from tensorflow.keras.layers import MaxPooling2D
from tqdm import tqdm
from PIL import Image
import cv2

In [4]:
from tensorflow.keras.preprocessing.image import load_img,img_to_array
def preprocess(imgpath):
    img=cv2.imread(imgpath)
    img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    img_black=cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    mask=img_black>5
    temp=img[np.ix_(mask.any(1),mask.any(0))]
    if temp.shape[0]==0:
        clean_img=img
    else:
        img0=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
        img1=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
        img2=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
        clean_img=np.stack([img0,img1,img2],axis=-1)
    image_final=cv2.addWeighted(clean_img,0.8,cv2.GaussianBlur(clean_img,(15,15),30),0.2,80)
    yield image_final

In [5]:
class SPP(tf.keras.layers.Layer):
    def __init__(self,pool_list=[1,2,4],**kwargs):
        super(SPP,self).__init__()
        self.pool_list=pool_list
        self.flatten=tf.keras.layers.Flatten()
        self.concat=tf.keras.layers.Concatenate()
    def get_config(self,):
        config = super().get_config().copy()
        config.update({'self.pool_list':[1,2,4],
                      'self.flatten':tf.keras.layers.Flatten(),
                      'self.concat':tf.keras.layers.Concatenate()})
        return config
        
    def call(self,inputs):
        shape=inputs.get_shape().as_list()
        self.output_list=[]
        for i in self.pool_list:
            pool=int(math.ceil(shape[1]/i))
            stride=int(math.floor(shape[1]/i))
            x=MaxPooling2D((pool,pool),strides=(stride,stride))(inputs)
            x=self.flatten(x)
            self.output_list.append(x)
        output=self.concat(self.output_list)
        return output
        

In [10]:
from tensorflow.keras.models import load_model
inf_model=load_model('../input/eye-diabetic-new/dense_spp.h5',custom_objects={'SPP':SPP})
test=pd.read_csv('../input/aptos2019-blindness-detection/test.csv')

In [17]:
test['diagnosis']=-1
for i in tqdm(range(test.shape[0])):
    imgpath='../input/aptos2019-blindness-detection/test_images/'+str(test.iloc[i,0])+'.png'
    image=next(preprocess(imgpath))
    img=cv2.resize(image,(900,900))
    img=img_to_array(img)/255.0
    img=np.expand_dims(img,axis=0)
    predictions=inf_model.predict(img)
    test.loc[i,'diagnosis']=np.argmax(predictions,axis=1)[0]

100%|██████████| 1928/1928 [06:13<00:00,  5.16it/s]


In [18]:
test.head()

,id_code,diagnosis
0,0005cfc8afb6,2
1,003f0afdcd15,3
2,006efc72b638,2
3,00836aaacf06,2
4,009245722fa4,2


In [19]:
test['diagnosis'].value_counts()

2    1071
0     388
3     303
1      91
4      75
Name: diagnosis, dtype: int64

In [20]:
test.to_csv('submission.csv',index=False)